In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

# Bayesian Optimization


In [2]:
# Import the class
from xopt import Xopt
from botorch.test_functions.multi_fidelity import AugmentedHartmann


c:\users\ryan roussel\documents\github\xopt\xopt\bayesian\optimize.py:254: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (ele.shape[-1] == len(vocs[vocs_type]),


The `Xopt` object can be instantiated from a JSON or YAML file, or a dict, with the proper structure.

Here we will make one

In [3]:
# Make a proper input file. 
YAML = """
xopt: {output_path: null, verbose: true}

algorithm:
  name: bayesian_optimization
  options:  
      n_initial_samples: 16
      n_steps: 6
      verbose: True
      generator_options:
          acquisition_function: custom_acq.acq
          use_gpu: False

simulation: 
  name: test_multi_fidelity
  evaluate: xopt.evaluators.test_multi_fidelity.evaluate

vocs:
  name: test_multi_fidelity
  description: null
  simulation: test_multi_fidelity
  templates: null
  variables:
    x1: [0, 1.0]
    x2: [0, 1.0]
    x3: [0, 1.0]
    x4: [0, 1.0]
    x5: [0, 1.0]
    x6: [0, 1.0]
    cost: [0, 1.0]                          ## NOTE: THIS IS REQUIRED FOR MULTI-FIDELITY OPTIMIZATION
  objectives:
    y1: 'MINIMIZE'
  linked_variables: {}
  constants: {a: dummy_constant}

"""
config = YAML

In [4]:
X = Xopt(config)
X

Loading config as text



            Xopt 
________________________________           
Version: 0.4.3+98.gfc15b12.dirty
Configured: True
Config as YAML:
xopt: {output_path: null, verbose: true, algorithm: cnsga}
algorithm:
  name: bayesian_optimization
  function: xopt.bayesian.algorithms.bayesian_optimize
  options:
    n_initial_samples: 16
    n_steps: 6
    verbose: true
    generator_options: {acquisition_function: !!python/name:custom_acq.acq '', use_gpu: false}
    custom_model: null
    restart_file: null
    initial_x: null
simulation:
  name: test_multi_fidelity
  evaluate: xopt.evaluators.test_multi_fidelity.evaluate
  options: {extra_option: abc}
vocs:
  name: test_multi_fidelity
  description: null
  simulation: test_multi_fidelity
  templates: null
  variables:
    x1: [0, 1.0]
    x2: [0, 1.0]
    x3: [0, 1.0]
    x4: [0, 1.0]
    x5: [0, 1.0]
    x6: [0, 1.0]
    cost: [0, 1.0]
  objectives: {y1: MINIMIZE}
  linked_variables: {}
  constants: {a: dummy_constant}
  constraints: {}

# Run BayesOpt

In [5]:
# Pick one of these
from concurrent.futures import ThreadPoolExecutor as PoolExecutor
#from concurrent.futures import ProcessPoolExecutor as PoolExecutor

executor = PoolExecutor()
# This will also work. 
#executor=None

In [6]:
# Change max generations
X.run(executor=executor)
results = X.results

Starting at time 2021-08-27T10:40:48-05:00
started running optimization with generator: <xopt.bayesian.generators.generator.BayesianGenerator object at 0x000001D780B3AAF0>
submitting initial candidates at time 2021-08-27T10:40:48-05:00
starting optimization loop
Model creation time: 0.096 s
Candidate generation time: 0.44 s
Candidate(s): tensor([[0.5986, 0.2744, 0.3984, 0.3156, 0.3382, 0.8192, 0.7050]],
       dtype=torch.float64)
submitting candidates at time 2021-08-27T10:40:49-05:00
Model creation time: 0.116 s
Candidate generation time: 0.393 s
Candidate(s): tensor([[0.5780, 0.2772, 0.3839, 0.3045, 0.3146, 0.8329, 0.7266]],
       dtype=torch.float64)
submitting candidates at time 2021-08-27T10:40:49-05:00
Model creation time: 0.147 s
Candidate generation time: 0.418 s
Candidate(s): tensor([[0.5655, 0.2793, 0.3753, 0.2990, 0.2990, 0.8413, 0.7394]],
       dtype=torch.float64)
submitting candidates at time 2021-08-27T10:40:50-05:00
Model creation time: 0.175 s
Candidate generation t

### Get highest fidelity global optimum

In [7]:
# create generator object
gen = MultiFidelityGenerator(X.vocs)

NameError: name 'MultiFidelityGenerator' is not defined

In [ ]:
# create model
model = create_multi_fidelity_model(results['variables'], results['corrected_objectives'], results['corrected_constraints'], X.vocs)

In [ ]:
## NOTE: we want to get the minimum evaluated at the highest fidelity -> make sure to use get_recommendation
rec = gen.get_recommendation(model)
problem = AugmentedHartmann(negate=False)
problem(rec) ## NOTE: the correct global minimum is -3.32237